# Solve a pipe-flow problem

**plan**:
- make an arbitrary networkx object with finite sinks and infinite sources
- solve using scipy
- adapt network i/o to neo4j/cypher
- adapt solver to neo4j/cypher

## An arbitrary networkx object with finite sinks and initinite sources

In [1]:
import networkx as nx
from functools import reduce

Can either I) minimise work done: $$\sum{P_{source,eq}*q_{eq}}$$

Or II) assume the pressure at all sources is the same, and just solve for flow. Use super-sinks and super-sources to first solve for pressure, then use root finder.

In [24]:
G = nx.Graph()

In [25]:
G.add_nodes_from(['A','B','C','D','E','F','G','H','I','J','K'])

In [26]:
G.add_edges_from([
    ('A','B',{'z':5}),
    ('B','C',{'z':10}),
    ('C','D',{'z':4}),
    ('D','E',{'z':5}),
    ('E','H',{'z':3}),
    ('D','G',{'z':6}),
    ('G','I',{'z':12}),
    ('J','I',{'z':11}),
    ('C','F',{'z':2}),
    ('F','J',{'z':3}),
    ('B','K',{'z':7}),
    ('J','K',{'z':9}),
    ('H','I',{'z':7}),
    
])

In [29]:
G_orig = G.copy()

In [30]:
nodes = [n for n,d in dict(G.degree).items() if d==2]

In [31]:
subgraph_components = [c for c in nx.connected_components(G.subgraph(nodes))]

In [32]:
for cc in subgraph_components:
    print (cc)
    print (G.edges(cc, data=True))

    #print ([(j[0],j[1]) for j in G.edges(cc,data=True)])
    #print (set(reduce(lambda x,y: [x[0],x[1]]+[y[0],y[1]], G.edges(cc,data=True)))-cc)
    #print (set([j for sub in G.edges(cc, data=True) for j in sub] )-cc)

{'F'}
[('F', 'C', {'z': 2}), ('F', 'J', {'z': 3})]
{'E', 'H'}
[('E', 'D', {'z': 5}), ('E', 'H', {'z': 3}), ('H', 'I', {'z': 7})]
{'K'}
[('K', 'B', {'z': 7}), ('K', 'J', {'z': 9})]
{'G'}
[('G', 'D', {'z': 6}), ('G', 'I', {'z': 12})]


In [33]:
def simplify(G, start_node, end_node):
    
    # get degree-2 nodes
    nodes = [n for n,d in dict(G.degree).items() if d==2]
    print ()
    
    # get component subgraphs based on 
    subgraph_components = [c for c in nx.connected_components(G.subgraph(nodes))]
    
    
    for cc in subgraph_components:
        print (cc)
        
        if (start_node not in cc) and (end_node not in cc):

        
            # for each subgraph, get the edges
            edges = list(G.edges(cc, data=True))
            print (edges)

            # get the start and end nodes
            startend_nodes = list(set(reduce(lambda x,y: [x[0],x[1]]+[y[0],y[1]], G.edges(cc,data=True)))-cc)

            # get series impedence
            z_eq = sum([e[2]['z'] for e in edges])
            print ('new edge',startend_nodes[0], startend_nodes[1],f'z={z_eq}')

            # add the equivalent edge to the graph or get parallel equivalent
            if G.has_edge(startend_nodes[0], startend_nodes[1]):
                z_eq = 1/(1/z_eq + 1/G.get_edge_data(startend_nodes[0],startend_nodes[1])['z'])
                G.get_edge_data(startend_nodes[0],startend_nodes[1])['z']=z_eq                
            else:
                G.add_edge(startend_nodes[0], startend_nodes[1],z=z_eq)

            # remove edges and nodes
            G.remove_nodes_from(cc) # also removes edges
        
    return G        

In [35]:
simplify(G,'A','K')

{'F'}
[('F', 'C', {'z': 2}), ('F', 'J', {'z': 3})]
new edge J C z=5
{'E', 'H'}
[('E', 'D', {'z': 5}), ('E', 'H', {'z': 3}), ('H', 'I', {'z': 7})]
new edge I D z=15
{'K'}
{'G'}
[('G', 'D', {'z': 6}), ('G', 'I', {'z': 12})]
new edge I D z=18


In [40]:
list(G.edges(data=True))

[('A', 'B', {'z': 5}),
 ('B', 'C', {'z': 10}),
 ('B', 'K', {'z': 7}),
 ('C', 'J', {'z': 4.112903225806451}),
 ('J', 'K', {'z': 9})]

In [38]:
1/(1/15+1/18)

8.181818181818182

In [39]:
simplify(G,'A','K')

{'K'}
{'I', 'D'}
[('I', 'J', {'z': 11}), ('I', 'D', {'z': 8.181818181818182}), ('D', 'C', {'z': 4})]
new edge J C z=23.18181818181818


In [42]:
1/(1/23.18+1/5)+10+9

23.112845990063875

In [48]:
G=simplify(G,'A','K')

{'J', 'K', 'C'}


In [49]:
list(G.edges(data=True))

[('A', 'B', {'z': 5}),
 ('B', 'C', {'z': 10}),
 ('B', 'K', {'z': 7}),
 ('C', 'J', {'z': 4.112903225806451}),
 ('J', 'K', {'z': 9})]